# Setup

## Environment

1. Create a virtual environment or similar (this was built with Python 3.10, but 3.11 should work too), and install `requirements.txt`:
    ```bash
    pip install -r requirements.txt
    ```
2. Setup Google Cloud Application Default Credentials (see [this doc](https://cloud.google.com/docs/authentication/provide-credentials-adc)).
3. Copy the `.env.template` file and set keys and other information as indicated.

## Connection Setup

Load the `.env` file into the Python environment:

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)  

True

Initialize VertexAI:

In [2]:
import os
import vertexai
vertexai.init(
    project=os.environ.get("GOOGLE_PROJECT_NAME"),
    location=os.environ.get("GOOGLE_LOCATION",'us-east1'),
)

Initialize LangChain VertexAI components:

In [3]:
from langchain_google_vertexai import VertexAI
from langchain_google_vertexai import ChatVertexAI
from langchain_google_vertexai import VertexAIEmbeddings
from vertexai.generative_models import GenerativeModel

llmModel = VertexAI(model_name=os.environ.get('GOOGLE_LLM','gemini-1.5-flash'))
chatModel = ChatVertexAI(model=os.environ.get('GOOGLE_LLM','gemini-1.5-flash'), temperature=0)
embedModel = VertexAIEmbeddings(model_name=os.environ.get('GOOGLE_EMBED_MODEL','multimodalembedding')) 
genModel = GenerativeModel(model_name=os.environ.get('GOOGLE_LLM','gemini-1.5-flash'))

Initialize Cassio (Astra DB)

In [4]:
import cassio
cassio.init(auto=True)

And establish the graph store:

In [5]:
from ragstack_langchain.graph_store import CassandraGraphStore     

SITE_PREFIX="travel_docs"
graph_store = CassandraGraphStore(
    embedModel,
    node_table=f"{SITE_PREFIX}_nodes")

# Create and Load LangChain `Document`s

The example `Tourbook.pdf` is fairly complex in structure, both digitally and visually. 

## Text

A variety of parsing tools such as Unstructured and Adobe ExtractAPI were attempted on `Tourbook.pdf` file, attempting with both file structure and OCR techniques, to no avail. The Vertex LLM was able to parse (with a fairly generic prompt), but unfortunately exited early as it determined it was repeating existing content and cited the URL of this!

In this notebook we are trying to demonstrate multi-modal embedding and retrieval, so the information was manually parsed, and put into the file `Tourbook.json`. This contains the first 24 pages minus the cover page, the table of contents, and a map on page 3.

Each paragraph will be linked to its closest (lowest) heading with a bi-directional link; this results in a graph linked through the headings.

In [6]:
from langchain_core.documents import Document
#from ragstack_knowledge_store.links import Link
from ragstack_langchain.graph_store.links import METADATA_LINKS_KEY, Link

from typing import List 
import json
from hashlib import sha256

def get_links(doc: Document) -> List[Link]:
    header_join_text = ' >> '

    # Extract headers and content
    headers = [key for key in doc.metadata if key.startswith('h')]
    sorted_headers = sorted(headers, key=lambda h: int(h[1:]))  # Ensure headers are processed in order h1, h2, h3...
    header_contents = [doc.metadata[h] for h in sorted_headers]
    
    # Create link tags based on parent headers if available
    links = []
    if len(sorted_headers) > 1:
        # Construct the full path from all parent headers, excluding the last element which is the current header
        links.append(Link.bidir(kind='heading', tag=header_join_text.join(header_contents[:-1])))

    links.append(Link.bidir(kind='heading', tag=header_join_text.join(header_contents)))
    return links

def create_and_link_documents(data):
    documents = []
    h1_dict = {}

    for entry in data:
        doc_metadata = entry['metadata']
        doc_metadata['id'] = sha256(entry['page_content'].encode('utf-8')).hexdigest()
        doc = Document(page_content=entry['page_content'], metadata=doc_metadata)        

        # Keep note of the H1 for each page; h1_dict will have the last H1 for each page.
        h1 = doc_metadata.get('h1')
        if h1:
            h1_dict[doc_metadata['page_number']] = h1

        links = get_links(doc)
        doc.metadata['links'] = links

        documents.append(doc)

    return documents, h1_dict

In [7]:
test_data = [
    {"metadata":{"mime_type":"text/plain", "page_number": 1, "h1":"Top header" }, "page_content":"h1. Top header\nSome text here."},
    {"metadata":{"mime_type":"text/plain", "page_number": 1, "h1":"Top header", "h2":"Sub-header" }, "page_content":"h2. Sub-header\nSome more text here."},
    {"metadata":{"mime_type":"text/plain", "page_number": 1, "h1":"Top header", "h2":"Sub-header", "h3":"Detailed header A" }, "page_content":"h3. Detailed header A\nA is a great letter."},
    {"metadata":{"mime_type":"text/plain", "page_number": 1, "h1":"Top header", "h2":"Sub-header", "h3":"Detailed header B" }, "page_content":"h3. Detailed header B\nB is even better."},
    {"metadata":{"mime_type":"text/plain", "page_number": 1, "h1":"Top header", "h2":"Sub-header", "h3":"Detailed header B", "h4":"Explaination" }, "page_content":"h4. Explanation\nAnd here is why!"},
    {"metadata":{"mime_type":"text/plain", "page_number": 1, "h1":"Second Top header" }, "page_content":"h1. Second Top header\nAnd now, for numbers..."}
]

test_documents, _ = create_and_link_documents(test_data)

# Print the documents and their links for verification
for doc in test_documents:
    print(doc)
    print(f"Document ID: {doc.metadata['id']}, Content: {doc.page_content}")
    if doc.metadata['links']:
        for link in doc.metadata['links']:
            print(f"    Linked to: {link.kind} '{link.tag}'")



page_content='h1. Top header\nSome text here.' metadata={'mime_type': 'text/plain', 'page_number': 1, 'h1': 'Top header', 'id': 'afae4c432514884e0ea7ef1bc73a899a215e03e29efabbd8e7b183092174afc6', 'links': [Link(kind='heading', direction='bidir', tag='Top header')]}
Document ID: afae4c432514884e0ea7ef1bc73a899a215e03e29efabbd8e7b183092174afc6, Content: h1. Top header
Some text here.
    Linked to: heading 'Top header'
page_content='h2. Sub-header\nSome more text here.' metadata={'mime_type': 'text/plain', 'page_number': 1, 'h1': 'Top header', 'h2': 'Sub-header', 'id': '56f7ae457c48dedf5e9156fd1f6b574ef0689d07eef35bad18318f110344b870', 'links': [Link(kind='heading', direction='bidir', tag='Top header'), Link(kind='heading', direction='bidir', tag='Top header >> Sub-header')]}
Document ID: 56f7ae457c48dedf5e9156fd1f6b574ef0689d07eef35bad18318f110344b870, Content: h2. Sub-header
Some more text here.
    Linked to: heading 'Top header'
    Linked to: heading 'Top header >> Sub-header'
page_

In [8]:
with open('Tourbook.json', 'r', encoding='utf-8') as file:
    text_data = json.load(file)

# Process the input data
text_documents, h1_dict = create_and_link_documents(text_data)

## Images

For images, we will use `PyMuPDF` to extract images from the document, `base64` encode the image, and create a `Document` referencing the appropriate H1 heading for the page. 

In [9]:
import pymupdf
import base64

doc = pymupdf.open('Tourbook.pdf')
image_documents = []

# page_index starts from 0, so these are actual pages 3, 5-23, but are numbered 1 and 3-21. 
pages_to_process = [2] + list(range(4, 23))  

for page_index in pages_to_process:
    page = doc[page_index]
    image_list = page.get_images()
    adjusted_page_number = page_index - 1

    # Iterate over the images on the page
    for image_index, img in enumerate(image_list, start=1):
        xref = img[0]
        pix = pymupdf.Pixmap(doc, xref) 
        if pix.n - pix.alpha > 3:
            pix = pymupdf.Pixmap(pymupdf.csRGB, pix)

        base64_image = base64.b64encode(pix.tobytes(output="png")).decode('utf-8')

        if adjusted_page_number % 2 == 0:  # If it's even
            page_spread = f"{adjusted_page_number}-{adjusted_page_number+1}"
        else:  # If it's odd
            page_spread = f"{adjusted_page_number-1}-{adjusted_page_number}"

        h1 = h1_dict[adjusted_page_number]
        link_h1 = Link.outgoing(kind='heading', tag=h1)
        doc_metadata = {
            "mime_type": "image",
            "mime_subtype": "png",
            "mime_encoding": "base64",
            "page_number": adjusted_page_number, 
            "page_spread": page_spread, 
            "image_index": image_index,
            "h1": h1, 
            "links" : [ link_h1 ],
            "id": sha256(base64_image.encode('utf-8')).hexdigest()
        }
        # Now in theory, langchain_google_vertex.embeddings.embed_image() calls ImageBytesLoader.load_bytes
        # which can take a base64 string, but that wasn't working...but this URI trick does work!
        image_doc = Document(page_content=f"data:image/png;base64,{base64_image}", metadata=doc_metadata)
        image_documents.append(image_doc)

## Load Knowledge Store

In [10]:
docs = []
ids = []
# GraphStore doesn't currently support metadata filtering, and LangChain doesn't deal well with
# retrieved multimedia. More on this later.
for doc in text_documents:#  + image_documents:
    docs.append(doc)
    ids.append(doc.metadata['id'])

    if len(docs) >= 50:
        print("saving batch")
        graph_store.add_documents(docs, ids=ids)
        docs.clear()
        ids.clear()

if docs:
    print("saving batch")
    graph_store.add_documents(docs, ids=ids)

saving batch
saving batch
saving batch
saving batch


# Query Examples

A fairly standard prompt template:

In [11]:
from langchain_core.prompts import ChatPromptTemplate

# The 'without sounding American' part of the prompt is simply trying to tone down the 'enthusiasm',
# resulting in output that is more widely paletable. Feel free to remove it!
template = """
You are a helpful travel agent bot. 

Answer the question based STRICTLY on the following context, and if the context is irrelvant to the question, the context
should be ignored. If that means there is no remaining relevant context, apologize for your lack of knowledge of the area. 
Do not fill in additional destinations and sites - if it is not in the context, it should not be mentioned.

You should provide answers to the travellers questions in a manner that encourages them to travel, without sounding American. 
Include specific details of things to see and do, so that you appear knowledgeable about the destination. Give a detailed itinerary.

CONTEXT:
{context}

QUESTION:
{question}
"""
prompt = ChatPromptTemplate.from_template(template)

Some helper methods:

In [12]:
from IPython.display import display, Markdown

def format_docs(docs):
    formatted = "\n\n".join(f"From {doc.metadata['content_id']}: {doc.page_content}" for doc in docs)
    return formatted

# Helper method to render markdown in responses to a chain.
def run_and_render(chain, question):
    result = chain.invoke(question)
    display(Markdown(result))

And now some queries. Searching the `KnowledgeStore` with a depth of `0` is equivalent to a vector search:

In [13]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Depth 0 doesn't traverses edges and is equivalent to vector similarity only.
vector_retriever = graph_store.as_retriever(search_kwargs={"depth": 0})

vector_rag_chain = (
    {"context": vector_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chatModel
    | StrOutputParser()
)

In [14]:
run_and_render(vector_rag_chain, "what can I do in London?")

I apologize, but the provided information is about travel in India and its neighboring countries. I do not have any information about London. 


## Text Search

In [15]:
QUESTION="What can I do in the golden triangle?"
run_and_render(vector_rag_chain, QUESTION)

The Golden Triangle is indeed the most popular tour of India, and for good reason!  It includes Delhi, the capital of India; Agra, home of the Taj Mahal; and Jaipur, the pink city of Rajasthan. 

Let me give you a possible itinerary:

First, you will fly into Delhi, where you can experience the hustle and bustle of India's capital city.  You can see the Red Fort, the Jama Masjid, and Humayun's Tomb.  You can also experience shopping in Old Delhi, which is an experience in itself!

Next, you will travel to Agra, home of the Taj Mahal.  The Taj Mahal is one of the Seven Wonders of the World, and is a must-see for any visitor to India.  You can also see the Agra Fort, which is another impressive Mughal fort.

Finally, you will travel to Jaipur, the "Pink City".  Jaipur is known for its beautiful pink architecture, and you can visit the Hawa Mahal, the City Palace, and the Amber Fort.  Jaipur is also a great place to buy souvenirs. 

This is just a suggestion, and the Golden Triangle can be easily customized to fit your interests.  For example, you could add on a visit to Ranthambore National Park to see tigers, or you could visit Neemrana for a more rustic experience.  No matter what you choose, you are sure to have an unforgettable time in India! 


One such example output (with `temperature=0`) is:

> The Golden Triangle is indeed the most popular tour of India, and for good reason! It covers three incredible cities: Delhi, Agra, and Jaipur.
> 
> You'll begin your journey in Delhi, the bustling capital of India. Here, you can experience the flavours of India's diverse street food scene, wander through the historic Red Fort, and marvel at the towering Qutub Minar.
> 
> Next, you'll travel to Agra, home of the breathtaking Taj Mahal. This monument to love, built entirely of white marble, is truly a sight to behold. While in Agra, don't miss the Agra Fort, another impressive Mughal fort with stunning views of the Taj.
> 
> Finally, you'll make your way to Jaipur, the "Pink City" of Rajasthan. Jaipur is a vibrant city with a rich history and culture. Explore the majestic Amber Fort, wander through the City Palace, and marvel at the intricate architecture of the Hawa Mahal.
> 
> The Golden Triangle offers a perfect blend of history, culture, and architecture. It's a journey you won't soon forget!

Based on documents:

From 29877647e4fefb1a2c6305bfe3f731bdbc628b03ca55c428791ae460fb9fbff5: Suggested Itineraries
- Classic Golden Triangle: Delhi - Agra - Jaipur - Delhi
- Golden Triangle with Tigers: Delhi - Jaipur - Ranthambore Bharatpur - Agra - Delhi
- Golden Triangle with Samode: Delhi - Agra - Samode - Jaipur - Delhi
- Golden Triangle with Neemrana: Delhi - Neemrana - Jaipur - Agra - Delhi
- Rustic Charm of Golden Triangle: Delhi - Mandawa - Jaipur - Agra - Delhi
- Images of North India: Delhi - Jaipur - Agra - Orchha / Khajuraho - Varanasi
- Grand Heritage of North India: Delhi - Jaipur - Agra - Gwalior Orchha / Khajuraho - Varanasi

From 26b4df804db61263d6f80f1654f6dd3b6fae2a531811cd26a5845e52def37516: Golden Triangle Extensions
Spice up the classic Golden Triangle tour by extending it to any the following interesting destinations

From 0fd018f03f0b1f078a2c1d64fc5048115921b22242e8e19f931d04c968af82cd: Golden Triangle
The most popular tour of India, the Golden Triangle tour, includes a visit to Delhi, the capital of India; Agra, land of the world famous Taj Mahal; and Jaipur, the pink capital city of Rajasthan.

From e6f649156034f99f24aec8782914dbf9e3db47881f72a519dbb0cdfd0dc2359e: River Cruises
ASSAM BENGAL NAVIGATION : Assam & Bengal
Experience the hidden beauty and treasures of remote north eastern regions of Assam and Bengal.


Experimenting with different temperartures increases 'hallucinations' or at least not strict adherence to the prompt, but even here we can find references to things not in the context, like "Qutub Minar" or "Agra Fort". While Agra Fort is referenced elsewhere in the travel document, Qutub Minar is not (though it is a sight one can see in Delhi).

Let's run the same question, but searching the graph with `depth=1`, which has the effect of searching for paragraphs within the same heading:

In [16]:
# Depth 1 does vector similarity and then traverses 1 level of edges.
graph_retriever = graph_store.as_retriever(search_kwargs={"depth": 1})

graph_rag_chain = (
    {"context": graph_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chatModel
    | StrOutputParser()
)
run_and_render(graph_rag_chain, QUESTION)

The Golden Triangle is indeed the most popular tour of India, and for good reason!  It allows visitors to experience India's captivating history and vibrant culture.  This journey will take you through Delhi, Agra, and Jaipur:

**Delhi**, the capital of India, is a bustling metropolis that seamlessly blends the old with the new. Here you can:

*   Explore Old Delhi's bustling streets and marvel at the majestic Jama Masjid, India's largest mosque.
*   Experience the flavours of Chandni Chowk, a vibrant market with delicious street food and spices.
*   Discover the serene beauty of Humayun's Tomb, a Mughal architectural masterpiece.
*   Visit Qutub Minar, a towering minaret and a UNESCO World Heritage Site.

**Agra**, home to the Taj Mahal, offers a glimpse into the Mughal era's grandeur. Here's what awaits you:

*   Witness the ethereal beauty of the Taj Mahal at sunrise, an unforgettable experience.
*   Explore the Agra Fort, a UNESCO World Heritage Site that showcases Mughal architecture.
*   Visit Itmad-ud-Daulah's Tomb, often called the "Baby Taj," for its intricate marble work.

**Jaipur**, the "Pink City," will enchant you with its vibrant colours and rich heritage. Here's what you can do:

*   Explore the Amber Fort, a majestic hilltop fort with stunning views.
*   Wander through the City Palace, a sprawling complex showcasing Jaipur's royal past.
*   Admire the intricate facade of Hawa Mahal, also known as the "Palace of Winds."
*   Shop for colourful textiles, jewellery, and handicrafts in Jaipur's bustling bazaars.

The Golden Triangle offers a diverse and unforgettable experience, combining history, culture, and architectural marvels. 


Sample output (`temperature=0`):

> The Golden Triangle is indeed a classic for a reason! It covers three incredible cities: Delhi, Agra, and Jaipur.
> 
> You'll start in Delhi, the bustling capital of India, where ancient history meets modern bustle. You can explore the Red Fort, a magnificent sandstone monument, and lose yourself in the labyrinthine alleys of Chandni Chowk, a shopper's paradise.
> 
> Next, you'll journey to Agra, home to the Taj Mahal. This white marble mausoleum is even more breathtaking in person! Don't miss the Agra Fort either, a UNESCO World Heritage site that offers a glimpse into Mughal grandeur.
> 
> Finally, you'll fall in love with Jaipur, the "Pink City". Here, you can wander through the majestic Amber Fort, marvel at the intricate facade of Hawa Mahal, and explore the City Palace, a stunning complex of courtyards and gardens.

Based on documents:

From 29877647e4fefb1a2c6305bfe3f731bdbc628b03ca55c428791ae460fb9fbff5: Suggested Itineraries
- Classic Golden Triangle: Delhi - Agra - Jaipur - Delhi
- Golden Triangle with Tigers: Delhi - Jaipur - Ranthambore Bharatpur - Agra - Delhi
- Golden Triangle with Samode: Delhi - Agra - Samode - Jaipur - Delhi
- Golden Triangle with Neemrana: Delhi - Neemrana - Jaipur - Agra - Delhi
- Rustic Charm of Golden Triangle: Delhi - Mandawa - Jaipur - Agra - Delhi
- Images of North India: Delhi - Jaipur - Agra - Orchha / Khajuraho - Varanasi
- Grand Heritage of North India: Delhi - Jaipur - Agra - Gwalior Orchha / Khajuraho - Varanasi

From 26b4df804db61263d6f80f1654f6dd3b6fae2a531811cd26a5845e52def37516: Golden Triangle Extensions
Spice up the classic Golden Triangle tour by extending it to any the following interesting destinations

From 0fd018f03f0b1f078a2c1d64fc5048115921b22242e8e19f931d04c968af82cd: Golden Triangle
The most popular tour of India, the Golden Triangle tour, includes a visit to Delhi, the capital of India; Agra, land of the world famous Taj Mahal; and Jaipur, the pink capital city of Rajasthan.

From e6f649156034f99f24aec8782914dbf9e3db47881f72a519dbb0cdfd0dc2359e: River Cruises
ASSAM BENGAL NAVIGATION : Assam & Bengal
Experience the hidden beauty and treasures of remote north eastern regions of Assam and Bengal.

From 4903ba59f1b56353140d0bc71432c00c83bb509877dc6af6cb52647668c1a7d7: Forts & Palaces of Rajasthan
India's forts and palaces are the country's treasured manuscripts narrating the saga of this Land of Maharajas. The desert state of Rajasthan is where history comes alive in the form of art, architecture, sculptures and cultures that brings thousands of heritage lovers to India.
Rajasthan houses the largest number of forts and palaces in the world. Today, some of these architectural marvels have been converted into heritage hotels, allowing you the luxury of a palatial stay in India.

From 4f9d948c96686135722c7d6635f7d8d4194b56dbe7bffad34caf188ec30856e8: Rural Rajasthan
In the heart of Rajasthan, lie a number of small towns and villages, which define the true essence of this land. A visit to these hidden gems, provides an opportunity to learn about the people, culture and lifestyle of rural India. Stay at erstwhile forts and palaces, which have now been converted to heritage hotels, and experience the intriguing traditions and customs of tribal Rajasthan.

From 9c4c48781277dca9942e58faa29c0d88e97cf01027b67f57819d1ff8e65991ba: NORTH INDIA
North India is a medley of kingdoms, cultures, traditions, architectural marvels and a spectrum of varied landscapes. Ranging from the snow capped Himalayan crown to the agriculturally rich plains of Punjab, the arid deserts of Rajasthan, the contrasting extremes of Old and New Delhi, the wonder of the world Taj Mahal, and the ancient cities of Haridwar and Varanasi set along the banks of holy River Ganges, North India offers a continuum of unique experiences.

From 9e8aad6fed1134dc8e97748c32f61b58f3e27ce4fcef4707f4b7ad4573dbe47d: Desert Camps of Rajasthan
Rajasthan, home to the golden sand dunes, is a haven for desert camps in India. For the erstwhile Maharajas, campaigning in distant lands meant long stays in desert camps. During wars or expeditions, camps were the only luxury available and soon these “homes away from home” evolved into mobile, mini-palaces.
Today, Rajasthan offers a wonderful selection of desert camps, to create the ideal nomadic desert retreat. Most camps offer camel and jeep safaris to nearby villages and evening entertainment with bon fire, local musicians & dancers, puppet shows, juggling, and other regional performances.

From f98dd1fb149fa011ee848fb9c640f682d8e23bcfa931009dd9e6e444aec0a71a: Himalayan Beauty
The Himalayas extend over 2,500km in east-west and between a range of 250-425km in north-south direction. Formed in three parallel ranges, the Himalayas extend over more than one country. Himalayan tours can be designed to suit diverse interests including adventure, culture, wellness, spiritual, wildlife, walking or leisure tours.

From 28461e5c87919540d2b2f7eeb2f42bad162f1f630de487bc68317fcca3b8c92f: JAIPUR is an exuberance of colours, folk dances, and traditional arts and crafts. This princely city is home to a wonderful collection of palatial masterpieces such as the City Palace, Amber Fort, Jal Mahal and Hawa Mahal. One of the most well planned cities of India, Jaipur is built in 9 rectangular sectors symbolising 9 divisions of the universe, as per Indian cosmology.

From 7cc9c87399cf23933625b69e09e375d2307fdbe21d78a3ec61b6df73b4d9ede8: AGRA, the Mughal city founded in the 16th century, is one of the most visited tourist city of India. Agra is home to magnificent architectural masterpieces such as the beautiful wonder of the world, Taj Mahal and the splendid Agra Fort. The artisans of Agra still practice the Mughal art of marble in-lay work, creating magical beauty with marble.

From d47bc13429d720944073b2deeaeddadeb4bf0962467a4977929111defe520634: DELHI, the power seat of India, is divided into two parts - Old and New Delhi. The walled city of Old Delhi narrates the history of the city, whilst the contemporary New Delhi reflects the cosmopolitan face of India. From the chaotic streets of Chandni Chowk to the imposing features of Lutyen's Delhi, this capital city encapsulates the extremes of historic and modern India.

From 2057ac35a9ac14137a9e6d52d4c727a5d3cd1cf5b888c61daa4eab67ad2012da: Cruising through the Holy River Ganges and Hugli, you get an opportunity to see unique cattle fairs of eastern India, holy temples, remains of the British Raj, and some historic Buddhist sites

From 9cced1f0bf264552bb4766042a4f0b31d91a479b0544135060aa6cacdaa2fef2: Enjoy tribal encounters and village walks, interact with the locals, discover tea plantations, explore country towns on cycle rickshaws, relish barbecues on deserted river islands, enjoy folk dance performances, visit temples, shrines and local craft workshops. This is your chance to experience the real rural India... still far from any signs of modernity.

From c720fbb857bc04e3b46b1c07be3488fb9a59594cb2e3cd34d9b482fe314d0c5e: Cruising through River Brahmaputra and Hugli, experience elephant and jeep safaris in some of India's most treasured wildlife parks- Manas, Kaziranga and Orang National Park.

From d9559a68e2fe4c00347435cf9f382b3e94a71174c829860f7fc40a2fb93ec35f: For detailed itineraries, departure dates, and river boat specifications, log on to our website www.traveliteindia.com

So it has a more complete itinrary and interesting detail, though it has still introduced something (the Red Fort) not in the context, albeit something legitimate to see in Delhi. What happens if we connect two edges away:

In [126]:
graph_retriever2 = graph_store.as_retriever(search_kwargs={"depth": 2})

graph_rag_chain2 = (
    {"context": graph_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chatModel
    | StrOutputParser()
)
run_and_render(graph_rag_chain2, QUESTION)

The Golden Triangle is indeed a classic tour for a reason!  It covers three incredible cities in India: Delhi, Agra, and Jaipur. 

In Delhi, you will be captivated by the blend of old and new.  Make sure to explore the bustling streets of Chandni Chowk in Old Delhi and the majestic Lutyen's Delhi.  

Next, make your way to Agra, home of the breathtaking Taj Mahal. This architectural masterpiece is truly a wonder of the world.  While you are there, don't miss the impressive Agra Fort, another example of Mughal splendor.  

Finally, your journey takes you to Jaipur, the vibrant Pink City.  Prepare to be amazed by the City Palace, the majestic Amber Fort, and the iconic Hawa Mahal.  Jaipur is a city that beautifully showcases the rich colours and culture of Rajasthan. 


Sample Output (`temperature=0`):

> The Golden Triangle is indeed a classic tour for a reason! It covers three incredible cities in India: Delhi, Agra, and Jaipur.
> 
> In Delhi, you will be captivated by the blend of old and new. Make sure to explore the bustling streets of Chandni Chowk in Old Delhi and the majestic Lutyen's Delhi.
> 
> Next, make your way to Agra, home of the breathtaking Taj Mahal. This architectural masterpiece is truly a wonder of the world. While you are there, don't miss the impressive Agra Fort, another example of Mughal splendor.
> 
> Finally, your journey takes you to Jaipur, the vibrant Pink City. Prepare to be amazed by the City Palace, the majestic Amber Fort, and the iconic Hawa Mahal. Jaipur is a city that beautifully showcases the rich colours and culture of Rajasthan.

Based on documents:

From 29877647e4fefb1a2c6305bfe3f731bdbc628b03ca55c428791ae460fb9fbff5: Suggested Itineraries
- Classic Golden Triangle: Delhi - Agra - Jaipur - Delhi
- Golden Triangle with Tigers: Delhi - Jaipur - Ranthambore Bharatpur - Agra - Delhi
- Golden Triangle with Samode: Delhi - Agra - Samode - Jaipur - Delhi
- Golden Triangle with Neemrana: Delhi - Neemrana - Jaipur - Agra - Delhi
- Rustic Charm of Golden Triangle: Delhi - Mandawa - Jaipur - Agra - Delhi
- Images of North India: Delhi - Jaipur - Agra - Orchha / Khajuraho - Varanasi
- Grand Heritage of North India: Delhi - Jaipur - Agra - Gwalior Orchha / Khajuraho - Varanasi

From 26b4df804db61263d6f80f1654f6dd3b6fae2a531811cd26a5845e52def37516: Golden Triangle Extensions
Spice up the classic Golden Triangle tour by extending it to any the following interesting destinations

From 0fd018f03f0b1f078a2c1d64fc5048115921b22242e8e19f931d04c968af82cd: Golden Triangle
The most popular tour of India, the Golden Triangle tour, includes a visit to Delhi, the capital of India; Agra, land of the world famous Taj Mahal; and Jaipur, the pink capital city of Rajasthan.

From e6f649156034f99f24aec8782914dbf9e3db47881f72a519dbb0cdfd0dc2359e: River Cruises
ASSAM BENGAL NAVIGATION : Assam & Bengal
Experience the hidden beauty and treasures of remote north eastern regions of Assam and Bengal.

From 28461e5c87919540d2b2f7eeb2f42bad162f1f630de487bc68317fcca3b8c92f: JAIPUR is an exuberance of colours, folk dances, and traditional arts and crafts. This princely city is home to a wonderful collection of palatial masterpieces such as the City Palace, Amber Fort, Jal Mahal and Hawa Mahal. One of the most well planned cities of India, Jaipur is built in 9 rectangular sectors symbolising 9 divisions of the universe, as per Indian cosmology.

From 7cc9c87399cf23933625b69e09e375d2307fdbe21d78a3ec61b6df73b4d9ede8: AGRA, the Mughal city founded in the 16th century, is one of the most visited tourist city of India. Agra is home to magnificent architectural masterpieces such as the beautiful wonder of the world, Taj Mahal and the splendid Agra Fort. The artisans of Agra still practice the Mughal art of marble in-lay work, creating magical beauty with marble.

From d47bc13429d720944073b2deeaeddadeb4bf0962467a4977929111defe520634: DELHI, the power seat of India, is divided into two parts - Old and New Delhi. The walled city of Old Delhi narrates the history of the city, whilst the contemporary New Delhi reflects the cosmopolitan face of India. From the chaotic streets of Chandni Chowk to the imposing features of Lutyen's Delhi, this capital city encapsulates the extremes of historic and modern India.

From 4903ba59f1b56353140d0bc71432c00c83bb509877dc6af6cb52647668c1a7d7: Forts & Palaces of Rajasthan
India's forts and palaces are the country's treasured manuscripts narrating the saga of this Land of Maharajas. The desert state of Rajasthan is where history comes alive in the form of art, architecture, sculptures and cultures that brings thousands of heritage lovers to India.
Rajasthan houses the largest number of forts and palaces in the world. Today, some of these architectural marvels have been converted into heritage hotels, allowing you the luxury of a palatial stay in India.

From 4f9d948c96686135722c7d6635f7d8d4194b56dbe7bffad34caf188ec30856e8: Rural Rajasthan
In the heart of Rajasthan, lie a number of small towns and villages, which define the true essence of this land. A visit to these hidden gems, provides an opportunity to learn about the people, culture and lifestyle of rural India. Stay at erstwhile forts and palaces, which have now been converted to heritage hotels, and experience the intriguing traditions and customs of tribal Rajasthan.

From 9c4c48781277dca9942e58faa29c0d88e97cf01027b67f57819d1ff8e65991ba: NORTH INDIA
North India is a medley of kingdoms, cultures, traditions, architectural marvels and a spectrum of varied landscapes. Ranging from the snow capped Himalayan crown to the agriculturally rich plains of Punjab, the arid deserts of Rajasthan, the contrasting extremes of Old and New Delhi, the wonder of the world Taj Mahal, and the ancient cities of Haridwar and Varanasi set along the banks of holy River Ganges, North India offers a continuum of unique experiences.

From 9e8aad6fed1134dc8e97748c32f61b58f3e27ce4fcef4707f4b7ad4573dbe47d: Desert Camps of Rajasthan
Rajasthan, home to the golden sand dunes, is a haven for desert camps in India. For the erstwhile Maharajas, campaigning in distant lands meant long stays in desert camps. During wars or expeditions, camps were the only luxury available and soon these “homes away from home” evolved into mobile, mini-palaces.
Today, Rajasthan offers a wonderful selection of desert camps, to create the ideal nomadic desert retreat. Most camps offer camel and jeep safaris to nearby villages and evening entertainment with bon fire, local musicians & dancers, puppet shows, juggling, and other regional performances.

From f98dd1fb149fa011ee848fb9c640f682d8e23bcfa931009dd9e6e444aec0a71a: Himalayan Beauty
The Himalayas extend over 2,500km in east-west and between a range of 250-425km in north-south direction. Formed in three parallel ranges, the Himalayas extend over more than one country. Himalayan tours can be designed to suit diverse interests including adventure, culture, wellness, spiritual, wildlife, walking or leisure tours.

From 2057ac35a9ac14137a9e6d52d4c727a5d3cd1cf5b888c61daa4eab67ad2012da: Cruising through the Holy River Ganges and Hugli, you get an opportunity to see unique cattle fairs of eastern India, holy temples, remains of the British Raj, and some historic Buddhist sites

From 9cced1f0bf264552bb4766042a4f0b31d91a479b0544135060aa6cacdaa2fef2: Enjoy tribal encounters and village walks, interact with the locals, discover tea plantations, explore country towns on cycle rickshaws, relish barbecues on deserted river islands, enjoy folk dance performances, visit temples, shrines and local craft workshops. This is your chance to experience the real rural India... still far from any signs of modernity.

From c720fbb857bc04e3b46b1c07be3488fb9a59594cb2e3cd34d9b482fe314d0c5e: Cruising through River Brahmaputra and Hugli, experience elephant and jeep safaris in some of India's most treasured wildlife parks- Manas, Kaziranga and Orang National Park.

From d9559a68e2fe4c00347435cf9f382b3e94a71174c829860f7fc40a2fb93ec35f: For detailed itineraries, departure dates, and river boat specifications, log on to our website www.traveliteindia.com

So much the same result...but also much the same documents.

In [127]:
mmr_graph_retriever = graph_store.as_retriever(
    search_type = "mmr_traversal",
    search_kwargs = {
        "k": 4,
        "fetch_k": 10,
        "depth": 2,
        # "score_threshold": 0.2,
    },
)

mmr_graph_rag_chain = (
    {"context": mmr_graph_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chatModel
    | StrOutputParser()
)

run_and_render(mmr_graph_rag_chain, QUESTION)

Ah, the Golden Triangle of India! A classic journey! You will be enchanted by the rich history and vibrant culture. 

I suggest you follow the classic Golden Triangle itinerary:

1. **Delhi**, the bustling capital city. Explore the Red Fort, a magnificent Mughal fort, and the Jama Masjid, one of India's largest mosques. Lose yourself in the labyrinthine alleys of Chandni Chowk, a shopper's paradise. 

2. **Agra**, home to the Taj Mahal, an eternal monument of love. Be mesmerised by its beauty. Visit Agra Fort, another impressive Mughal fort, and Fatehpur Sikri, a deserted city frozen in time.

3. **Jaipur**, the Pink City. Explore the majestic Amber Fort,  be captivated by the intricate facade of Hawa Mahal, and wander through the City Palace for a glimpse into royal history. 

This journey will leave you with unforgettable memories. 


Based on documents:

From 29877647e4fefb1a2c6305bfe3f731bdbc628b03ca55c428791ae460fb9fbff5: Suggested Itineraries
- Classic Golden Triangle: Delhi - Agra - Jaipur - Delhi
- Golden Triangle with Tigers: Delhi - Jaipur - Ranthambore Bharatpur - Agra - Delhi
- Golden Triangle with Samode: Delhi - Agra - Samode - Jaipur - Delhi
- Golden Triangle with Neemrana: Delhi - Neemrana - Jaipur - Agra - Delhi
- Rustic Charm of Golden Triangle: Delhi - Mandawa - Jaipur - Agra - Delhi
- Images of North India: Delhi - Jaipur - Agra - Orchha / Khajuraho - Varanasi
- Grand Heritage of North India: Delhi - Jaipur - Agra - Gwalior Orchha / Khajuraho - Varanasi

From 26b4df804db61263d6f80f1654f6dd3b6fae2a531811cd26a5845e52def37516: Golden Triangle Extensions
Spice up the classic Golden Triangle tour by extending it to any the following interesting destinations

From d820c47547490681c04ababf7d66397bcd570c62fd5c9dfd280ab16f990c3d19: When travelling to India, it is worth taking a short trip to a neighbouring country, to enhance your experience. You will notice a striking similarity, yet subtle differences in the people, food, culture, religion and lifestyle in the entire subcontinent. It is in these differences that we celebrate the 'unity in diversity of the Indian subcontinent.

From 14345787c02a3d3eedf901803acce8ba4d1e6ef86470355884d8149129b467c5: LUMBINI (Nepal), the celebrated birthplace of Lord Buddha.

No idea what it's done here on the MMR retriever, but it's very little  to work from so it's filled in a lot of detail from outside the context...

## Search with Image + Text

This image is not in the brochure:

![](camel-riding.jpg)!

In [129]:
from langchain_core.messages import HumanMessage

image_message = {
    "type": "image_url",
    "image_url": {"url":"camel-riding.jpg"},
}

text_message = {
    "type": "text",
    "text": "Where can I do this while on my trip around the golden triangle?",
}
message = HumanMessage(content=[text_message, image_message])

output = chatModel.invoke([message])
display(Markdown(output.content))


Camel riding is a popular activity in **Rajasthan, India**, which is where the Golden Triangle tourist circuit is located. 

Within the Golden Triangle, the most popular place for camel safaris is **Pushkar**. Pushkar is a holy city known for its annual camel fair, but you can enjoy camel rides there year-round. 

You can also find camel riding opportunities in **Jaipur**, particularly near the Amber Fort and in the surrounding desert areas. 

Remember to choose ethical tour operators who prioritize the well-being of their animals. 


In [132]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage
from langchain_core.language_models import BaseChatModel

def describe_image(llm: BaseChatModel, image_url: str, instruct: str) -> str:
    """Describe the contents of an image."""
    system_message = SystemMessage(content="""
        You are a tool that converts images to text.     
    """)

    image_message = HumanMessage(content=[
    {"type": "text", "text": instruct or "Describe the contents of this image."},
    {"type": "image_url","image_url": {"url":image_url}}
    ])
    
    output = llm.invoke([system_message, image_message])
    return output.content
    
image_description = describe_image(chatModel,"camel-riding.jpg","Describe the image as completely as possible in 50-80 words, in a manner that is suitable for searching travel brochures.")
IMAGE_QUESTION=f"Image: {image_description}\n Can you include this on an iterary that also includes the Golden Triangle?"
run_and_render(graph_rag_chain2, IMAGE_QUESTION)

Absolutely! A camel safari through the majestic sand dunes sounds like an excellent addition to your Indian adventure! 

We can easily include this as part of your Golden Triangle tour.  This popular tour allows you to experience three of India's most captivating destinations: Delhi, Agra, and Jaipur. 

Let me paint you a picture of your itinerary:

1. **Delhi:** Your journey begins in Delhi, India's vibrant capital. Explore the bustling streets of Old Delhi, with its historic mosques and bustling bazaars.  Marvel at the architectural grandeur of Humayun's Tomb and Qutub Minar. In New Delhi, contrast the wide avenues and colonial-era buildings, a testament to India's rich history.

2. **Agra:**  Next, we journey to Agra, home to the breathtaking Taj Mahal. Witness the ethereal beauty of this ivory-white marble mausoleum, an enduring symbol of love. Explore the Agra Fort, a UNESCO World Heritage site, and be captivated by its impressive Mughal architecture.

3. **Jaipur:**  Prepare to be enchanted by Jaipur, the "Pink City" of Rajasthan. Immerse yourself in the grandeur of the Amber Fort, where you can even ascend on elephant back.  Wander through the City Palace, a sprawling complex showcasing Rajput architecture.  And of course, no trip to Jaipur is complete without a visit to the Hawa Mahal, or "Palace of Winds," with its intricate facade.

4. **Desert Adventure:**  Here's where your unforgettable camel safari comes in! From Jaipur, we'll whisk you away to the enchanting desert landscapes of Rajasthan.  Picture this:  as the sun sets, casting a golden glow over the dunes, you'll mount your gentle camel and embark on an unforgettable journey.  Experienced cameleers will guide you through the undulating landscape, sharing their knowledge of the desert and its secrets.  As you traverse the golden expanse, you'll witness breathtaking views and experience the true essence of desert life.  Imagine the tranquility of the desert night, as you gather around a crackling bonfire, enjoying traditional Rajasthani music and cuisine under a starlit sky.

This itinerary offers a perfect blend of cultural immersion and natural beauty, creating memories to last a lifetime.  Would you like to explore this option further? 


From documents:

From 8df175fc1cc741e51c545d8d12e62ace4787f01747796d8505432c4f45573fe7: JAISALMER, the golden city of Rajasthan and a haven for desert camps offers a wonderful selection of camps to suit every budget. Popular desert camps in and around Jaisalmer include: 
- The Serai, a luxury desert camp and spa, offers 21 tents, including a 'Royal Suite' with its private pool, spa and lounging tent and 6 'Luxury Tents' with private walled gardens and plunge pools. 
- Mirvana Nature Resort, located midway between Pokhran and Jaisalmer, provides 15 luxury tents, raised on a marble platform, crafted using local natural materials. Being a nature resort, Mirvana also offers organic food and fresh farm dairy. 
- Mool Sagar Camp, a historic camp with 18 luxurious tents, delightful stone pavilions and fountains, and a swimming pool, set amidst wonderful gardens. 
- Dune Safari Camp, in the sand dunes of Sam, consists of 20 deluxe tents, with a rustic charm. The camp can also arrange desert marathons and desert survival lessons for small groups.

From 9e8aad6fed1134dc8e97748c32f61b58f3e27ce4fcef4707f4b7ad4573dbe47d: Desert Camps of Rajasthan
Rajasthan, home to the golden sand dunes, is a haven for desert camps in India. For the erstwhile Maharajas, campaigning in distant lands meant long stays in desert camps. During wars or expeditions, camps were the only luxury available and soon these “homes away from home” evolved into mobile, mini-palaces.
Today, Rajasthan offers a wonderful selection of desert camps, to create the ideal nomadic desert retreat. Most camps offer camel and jeep safaris to nearby villages and evening entertainment with bon fire, local musicians & dancers, puppet shows, juggling, and other regional performances.

From e5a09cfb00e1d277177bc5ab01209a85256d0095c119263e0cf7f045df2fffc4: Allow us to create an experience of a lifetime, an inspiring journey to an exotic land, discovering the people, culture, tradition, history and legacy of our Incredible India and its neighbouring wonderlands. With our passion, experience and meticulous planning, we promise to weave memories you will cherish forever.

From 9cced1f0bf264552bb4766042a4f0b31d91a479b0544135060aa6cacdaa2fef2: Enjoy tribal encounters and village walks, interact with the locals, discover tea plantations, explore country towns on cycle rickshaws, relish barbecues on deserted river islands, enjoy folk dance performances, visit temples, shrines and local craft workshops. This is your chance to experience the real rural India... still far from any signs of modernity.

From 1d97d3c07ddd009f2bc1a4d5f882b9785e9d2e77c6d964b754f3e22400b1841c: Welcome aboard your Indian voyage with Travelite (India)

From 51f57a627e579114432573624699c8011c7e81f439b50875e7b253e896f19cd9: India is a medley of fascinating colours and cultures, an historical legacy, a canvas of architectural masterpieces, and an extravagant exuberance of royal splendour. This land of sages and ancient legends is home to the eternal symbol of love - the Taj Mahal, magnificent forts and palaces of Rajasthan, emerald beaches of Goa and Kerala, ancient caves of Ajanta and Ellora and the Golden Temple of Amritsar.

From 795bc833b145bf2bb7bb409e7fd680ebeb8ac3498739b513274aa8af686f9466: In this brochure, we introduce you to the mesmerising beauty of the Indian subcontinent. Combining the cultural splendours of India with the snow covered Himalayan peaks of Nepal, the serenity of Bhutan, green tea estates of Sri Lanka and the turquoise beaches of Maldives, the Indian sub continent treasures it all.

From c07d382d6ef0372001e9deeff943efe7fd73307d67e5e5f0e40dbab7c2ce4e7f: Crowned by Himalayas in the north and surrounded by oceans in the south, India offers a platter full of variety, packed in one country. Ranging from walking trails through Himalayan villages, to desert camps in the sand dunes, tribal and rural experiences, modern metropolitan cities, wildlife parks, serene backwaters, luxury trains, river cruises and adventure tours, the choice is only limited by your imagination!

From d820c47547490681c04ababf7d66397bcd570c62fd5c9dfd280ab16f990c3d19: When travelling to India, it is worth taking a short trip to a neighbouring country, to enhance your experience. You will notice a striking similarity, yet subtle differences in the people, food, culture, religion and lifestyle in the entire subcontinent. It is in these differences that we celebrate the 'unity in diversity of the Indian subcontinent.

From 09c532e07336c3d177e96d351214ae062787af0dc2994c73540f4f6ad56427ea: MANDAWA, a quaint town in the heart of Shekhawati region in Rajasthan, offers rural charm in the form of mud cottages at the Desert Resort and spacious swiss tents at Camp Mandawa.

From 104cb1e1a9dc8c9357d4b4c6a78fb0b30117245ac4e2347b6443b7715dc240de: MANVAR, a tented camp amidst the wilderness surrounded by sand dunes, is an oasis of rare exclusivity. 30 safari-style tents arranged in a semi-circle around a central durbar hall, allow for commanding views over the desert wilderness. These luxurious tents are more like minimarquees with polished teak furniture, rich jute rugs and lovely custom designed block-printed tent linings.

From 29a3446c97ac5ff7ceedd8e17be7f0035363a0156439e7d458abf1e66b25428e: ROHETGARH, located close to Jodhpur, is a wilderness camp, with large tents furnished with typical Rajasthani decor. The tents are set up around a central lounge and dining area, which is beautifully relaxing by day and romantically lit with candles by night. The camp is located on a small dune, with spectacular views.

From 2c6c8815d0000adab0ed63cf5af244fc2d1f700e705272c400828a874b697399: PUSHKAR, is the hub of India's largest annual camel fair held in November every year. Pushkar is home to a beautiful collection of desert camps, ranging from the luxurious tents at Greenhouse and Aaram Bagh to the deluxe tents at Pushkar Bagh, which swing to life with people and large flocks of camel every November.

From 3877d5c5417dc8b7ac6f47fecfef31a3a3c96db435590c3d43233b19acea2e16: OSIAN, is a famous landmark camp in the sand dunes of Rajasthan. Featuring a refreshing swimming pool, the camp is a feat of architectural engineering in the sand dunes. Osian camp is famous for its safari bar, a wonderful place to relax in the evenings with a drink and cigar.

From 51acb5fbcbd452836c237ebd163765e6a115a25d39995a14e83da6b15206a849: RANTHAMBORE, the wildlife hub of Rajasthan, is home to a beautiful collection of desert camps ranging from the luxurious tents at Oberoi Vanyavilas and Aman-i-Khas, to the eco friendly lavish tents and cottages at Sherbagh and Khemvilas, deluxe cottages at Tiger Den and Pugmark, and the budget tents at Ranthambore Bagh.

From 74620a1a476e369fe3918b005d0a9215fc21c67539de5bc268eea136f5f7c20d: ABHANERI, the historic site of 7th century step wells, offers rustic camps as an ideal stop over en-route the famous golden triangle. The camp offers 12 deluxe tents with wall to wall carpets, mesh windows, wood and wrought iron furniture.

From 797a8725e0a9503c53c23b6b3457751f2c0984a3e79759a61d2fa8d714fe316a: NIMAJ, a small village in central Rajasthan is home to the luxurious Chhatra Sagar Camp, set along the banks of a water reservoir. Each tent is carefully hand stitched with interiors block printed in traditional floral motifs. The camp has 11 tents along the banks of the lake and 2 tents atop a hill providing complete privacy and breathtaking 360 degree views.

From a326c6231af912030be201728eb92ad4c31772c28ab327c285b1f43190a009c6: NAGAUR, like Pushkar, is home to an annual cattle fair, held in February every year. Founded in the 4th century, the city of Nagaur is dominated by the Nagaur Fort complex, which encompasses luxury tents, using the finest of Marwar traditional skills.

From 0fd018f03f0b1f078a2c1d64fc5048115921b22242e8e19f931d04c968af82cd: Golden Triangle
The most popular tour of India, the Golden Triangle tour, includes a visit to Delhi, the capital of India; Agra, land of the world famous Taj Mahal; and Jaipur, the pink capital city of Rajasthan.

From 4903ba59f1b56353140d0bc71432c00c83bb509877dc6af6cb52647668c1a7d7: Forts & Palaces of Rajasthan
India's forts and palaces are the country's treasured manuscripts narrating the saga of this Land of Maharajas. The desert state of Rajasthan is where history comes alive in the form of art, architecture, sculptures and cultures that brings thousands of heritage lovers to India.
Rajasthan houses the largest number of forts and palaces in the world. Today, some of these architectural marvels have been converted into heritage hotels, allowing you the luxury of a palatial stay in India.

From 4f9d948c96686135722c7d6635f7d8d4194b56dbe7bffad34caf188ec30856e8: Rural Rajasthan
In the heart of Rajasthan, lie a number of small towns and villages, which define the true essence of this land. A visit to these hidden gems, provides an opportunity to learn about the people, culture and lifestyle of rural India. Stay at erstwhile forts and palaces, which have now been converted to heritage hotels, and experience the intriguing traditions and customs of tribal Rajasthan.

From 9c4c48781277dca9942e58faa29c0d88e97cf01027b67f57819d1ff8e65991ba: NORTH INDIA
North India is a medley of kingdoms, cultures, traditions, architectural marvels and a spectrum of varied landscapes. Ranging from the snow capped Himalayan crown to the agriculturally rich plains of Punjab, the arid deserts of Rajasthan, the contrasting extremes of Old and New Delhi, the wonder of the world Taj Mahal, and the ancient cities of Haridwar and Varanasi set along the banks of holy River Ganges, North India offers a continuum of unique experiences.

From f98dd1fb149fa011ee848fb9c640f682d8e23bcfa931009dd9e6e444aec0a71a: Himalayan Beauty
The Himalayas extend over 2,500km in east-west and between a range of 250-425km in north-south direction. Formed in three parallel ranges, the Himalayas extend over more than one country. Himalayan tours can be designed to suit diverse interests including adventure, culture, wellness, spiritual, wildlife, walking or leisure tours.

From 2057ac35a9ac14137a9e6d52d4c727a5d3cd1cf5b888c61daa4eab67ad2012da: Cruising through the Holy River Ganges and Hugli, you get an opportunity to see unique cattle fairs of eastern India, holy temples, remains of the British Raj, and some historic Buddhist sites

From c720fbb857bc04e3b46b1c07be3488fb9a59594cb2e3cd34d9b482fe314d0c5e: Cruising through River Brahmaputra and Hugli, experience elephant and jeep safaris in some of India's most treasured wildlife parks- Manas, Kaziranga and Orang National Park.

From d9559a68e2fe4c00347435cf9f382b3e94a71174c829860f7fc40a2fb93ec35f: For detailed itineraries, departure dates, and river boat specifications, log on to our website www.traveliteindia.com

From e6f649156034f99f24aec8782914dbf9e3db47881f72a519dbb0cdfd0dc2359e: River Cruises
ASSAM BENGAL NAVIGATION : Assam & Bengal
Experience the hidden beauty and treasures of remote north eastern regions of Assam and Bengal.